<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Some-data-to-fit" data-toc-modified-id="Some-data-to-fit-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Some data to fit</a></span></li><li><span><a href="#RBF-kernel-definition" data-toc-modified-id="RBF-kernel-definition-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>RBF kernel definition</a></span></li><li><span><a href="#Fit-the-RBF-regression" data-toc-modified-id="Fit-the-RBF-regression-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Fit the RBF regression</a></span></li><li><span><a href="#Play-with-hyper-parameters-of-the-model" data-toc-modified-id="Play-with-hyper-parameters-of-the-model-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Play with hyper parameters of the model</a></span></li></ul></div>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
import rbfregression.core as rbfreg

In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib notebook

## Some data to fit

In [ ]:
def function(x, rand =.2):
    y = np.exp(-((x-50)/20.)**2)/2 + 1/150.*x
    y += rand * np.random.randn(len(x))
    return y

# Real function values
X_true = np.arange(0, 150, 1.)
Y_true = function(X_true, rand=0)

# Key points
X_k = np.arange(10, 141, 20.) 


# Data for learning
X_l = np.arange(0, 150, 1.) 
Y_l = function(X_l)

X_k = np.atleast_2d(X_k).T
X_l = np.atleast_2d(X_l).T


plt.figure()
plt.plot(X_true, Y_true, ':g', label ='true function')
plt.plot(X_l, Y_l, '.b', label ='noisy data for learning')
plt.grid()
plt.legend()
plt.show()

plt.savefig('Data.png')

## RBF kernel definition

In [ ]:
# RbF kernel
def TPS_kernel(r, radius):
    
    r[r==0]=1 
    return r * r * np.log(r)


def Gauss_kernel(r, radius):
    radius *= 0.707
    result = np.exp(-(r **2 ) / (2.0 * radius**2)) 
    return result


## Fit the RBF regression

In [ ]:
# Create the RegularizedRBF instance
RBF = rbfreg.RegularizedRBF(kernel=Gauss_kernel)
# Set the control points (key points)
RBF.set_key_points(X_k)
# Compute the fit
RBF.fit(X_l, Y_l, regularization=0.1, radius = 25)

# Eval the model at points
Y_s = RBF.eval(X_l)
Y_k = RBF.eval(X_k)

# Plot them all
plt.figure()
plt.plot(X_true, Y_true, ':g', label ='true function')
plt.plot(X_l, Y_l, '.b', label ='noisy data for learning')
plt.plot(X_l, Y_s, '-r', label ='RBF model')
plt.plot(X_k, Y_k, 'ok', label ='RBF model - control points')
plt.grid()
plt.legend()
plt.savefig('Exemple_1D.png')

## Play with hyper parameters of the model

In [ ]:
import ipywidgets as widgets
from ipywidgets import HBox, VBox

Y_l = function(X_l[:,0], rand=.1)

plt.figure()

plt.plot(X_l, Y_l, '.b', label ='noisy data for learning')
plt.plot(X_true, Y_true, ':g', label ='true function')
rbf_model,  = plt.plot([], [], '-r', label ='RBF model')
key_points, = plt.plot([], [], 'ok', label ='RBF model - control points')
plt.grid()
plt.legend()


@widgets.interact(regularization=(0,1.), radius=(1,100))
def plot_oscil_1(regularization=0, radius=25):
      
    RBF = rbfreg.RegularizedRBF(kernel=Gauss_kernel)
    RBF.set_key_points(X_k)
    RBF.fit(X_l, Y_l, regularization=regularization, radius = radius)
    Y_s = RBF.eval(X_l)
    Y_k = RBF.eval(X_k)

    rbf_model.set_data(X_l, Y_s)
    key_points.set_data(X_k, Y_k)

    